# Automatic detection of rest/activity periods with pyActigraphy

The **_pyActigraphy_** package implements several rest/activity detection algorithms.

Broadly speaking, they can be divided in two categories:

* Epoch-by-epoch rest/activity scoring (inspired by PSG):
    * Cole-Kripke's, Sadeh's, Scripps' and Oakley's algorithms
* Detection of consolidated periods of similar activity patterns
    * Crespo's, Roenneberg's algorithms

All algorithms have been implemented to return a binary time serie (0 being rest or activity depending on the definition made in the original article...)

In [ ]:
import pyActigraphy

In [ ]:
import plotly.graph_objs as go

In [ ]:
import os
fpath = os.path.join(os.path.dirname(pyActigraphy.__file__),'tests/data/')

In [ ]:
raw = pyActigraphy.io.read_raw_rpx(
    fpath+'test_sample_rpx_eng.csv', start_time='2015-07-04 12:00:00', period='6 days', language='ENG_UK'
)

## Epoch-by-epoch rest/activity scoring algorithms

All based on the same idea;

* use a rolling window over the data and convolute them with a "gaussian"-like kernel.
* if the resulting data is above a predefined threshold, classify as activity. Rest otherwise...


### My opinion regarding these algorithms
The weights of these "gaussian"-like kernels as well as the applied thresholds are very much device and population dependant. Using these algorithms would thus require to adapt these parameters to the population under study and the actigraph device... Possible but seldom done.

In [ ]:
layout = go.Layout(title="Rest/Activity detection",xaxis=dict(title="Date time"), yaxis=dict(title="Counts/period"), showlegend=False)

### Cole-Kripke

In [ ]:
help(raw.CK)

In [ ]:
CK = raw.CK()

In [ ]:
layout.update(yaxis2=dict(title='Classification',overlaying='y',side='right'), showlegend=True);

In [ ]:
go.Figure(data=[
    go.Scatter(x=raw.data.index.astype(str),y=raw.data, name='Data'),
    go.Scatter(x=CK.index.astype(str),y=CK, yaxis='y2', name='CK')
], layout=layout)

### Sadeh's and Scripps' algorithms

In [ ]:
sadeh = raw.Sadeh()
scripps = raw.Scripps()

In [ ]:
go.Figure(data=[
    go.Scatter(x=raw.data.index.astype(str),y=raw.data, name='Data'),
    go.Scatter(x=sadeh.index.astype(str),y=sadeh, yaxis='y2', name='Sadeh'),
    go.Scatter(x=scripps.index.astype(str),y=scripps, yaxis='y2', name='Scripps')
], layout=layout)

### Oakley's algorithm

This is the sleep/wake scoring algorithm used by the Actiware software from Respironics. The various activity thresholds have thus been tailored for Actiwatch devices. Be careful when applying this algorithm on data acquired with other devices.

In [ ]:
help(raw.Oakley)

#### Medium threshold
A threshold of 40 corresponds to a *medium* wake threshold.

In [ ]:
oakley = raw.Oakley(threshold=40) 

#### Automatic threshold
It is also possible to compute a threshold value automatically, based on activity data.

In [ ]:
oakley_auto = raw.Oakley(threshold='automatic')

In [ ]:
go.Figure(data=[
    go.Scatter(x=raw.data.index.astype(str),y=raw.data, name='Data'),
    go.Scatter(x=oakley.index.astype(str),y=sadeh, yaxis='y2', name='Oakley (thr: medium)'),
    go.Scatter(x=oakley_auto.index.astype(str),y=scripps, yaxis='y2', name='Oakley (thr: automatic)')
], layout=layout)

## Consolidated activity/rest period detection

### Crespo's algorithm

This is a threshold-based algorithm that used morphological filters to "clean" short periods of activity (rest) surrounded by periods of rest (acitivity).

In [ ]:
help(raw.Crespo)

In [ ]:
crespo = raw.Crespo()

In [ ]:
crespo_6h = raw.Crespo(alpha='6h')

In [ ]:
crespo_zeta = raw.Crespo(estimate_zeta=True)

In [ ]:
go.Figure(data=[
    go.Scatter(x=raw.data.index.astype(str),y=raw.data, name='Data'),
    go.Scatter(x=crespo.index.astype(str),y=crespo, yaxis='y2', name='Crespo'),
    go.Scatter(x=crespo_6h.index.astype(str),y=crespo_6h, yaxis='y2', name='Crespo (6h)'),
    go.Scatter(x=crespo_zeta.index.astype(str),y=crespo_zeta, yaxis='y2', name='Crespo (Automatic)')
], layout=layout)

Like for the other algorithms, the default parameters correspond to those described in the original papers, which, most likely, have been tuned to the population used to validate the algorithm.

Since the output is made of consolidated periods, it is possible to return the offset and onset times of each period:

In [ ]:
aot = raw.Crespo_AoT()

In [ ]:
aot

In [ ]:
aot[0]-aot[1]

### Roenneberg's algorithm

Also threshold-based but uses correlations with test series of various lengths to find the consolidated period that best matches best the data.

In [ ]:
help(raw.Roenneberg)

In [ ]:
roenneberg = raw.Roenneberg()
roenneberg_thr = raw.Roenneberg(threshold=0.25, min_seed_period='15min')

In [ ]:
go.Figure(data=[
    go.Scatter(x=raw.data.index.astype(str),y=raw.data, name='Data'),
    go.Scatter(x=roenneberg.index.astype(str),y=roenneberg, yaxis='y2', name='Roenneberg'),
    go.Scatter(x=roenneberg_thr.index.astype(str),y=roenneberg_thr, yaxis='y2', name='Roenneberg (Thr:0.25)')
], layout=layout)

In [ ]:
aot = raw.Roenneberg_AoT(threshold=0.25, min_seed_period='15min')

In [ ]:
aot

In [ ]:
aot[0]-aot[1]

Et voilà! Easy, isn't it?